In [5]:
import dask.dataframe as dd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from scipy.stats import zscore
import csv
from pathlib import Path
import pandas as pd

#DaskDf = pd.read_parquet('C:/Users/bincl/BA-Thesis/BA/HC/bd.gz.parquet')
DaskDf = dd.read_parquet('C:/Users/bincl/BA-Thesis/BA/HC/bd.gz.parquet')
#DaskDf = dd.read_parquet('C:/Users/bincl/BA-Thesis/BA/HC/bd.gz.parquet', header=0, sep='\t', quotechar='"', on_bad_lines='skip', quoting=csv.QUOTE_NONE)
    
print(DaskDf)    

KeyError: "An error occurred while calling the read_parquet method registered to the pandas backend.\nOriginal Message: 'None of [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None] are in the columns'"

In [ ]:
DaskDf.npartitions()

TypeError: 'int' object is not callable

In [ ]:
#pandaDf = pd.read_csv('C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_aa.gz', compression='gzip', header=0, sep='\t', quotechar='"', on_bad_lines='skip', quoting=csv.QUOTE_NONE)
#print(pandaDf)   

In [ ]:
df1gram = dd.read_csv('C:/Users/bincl/BA-Thesis/Dataset/1gram/1_20000_nopos.gz', compression='gzip', header=0, sep='\t', quotechar='"', on_bad_lines='skip', quoting=csv.QUOTE_NONE)
df2gram = DaskDf



c:\Users\bincl\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\csv.py:546: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


In [ ]:
def getLowerLevelString(string):
   first, *middle, last = string.split()
   first = first + ''.join(middle)
   last = ''.join(middle) + last
   return first, last

In [ ]:
def MLR(token):
    first,last = getLowerLevelString(token)
    X = dd.concat([df1gram.loc[first],df1gram.loc[last]],axis=1)
    y = df2gram.loc[token]
    reg = LinearRegression()
    reg.fit(X, y) 
    
    return reg.coef_,reg.intercept_,X

In [ ]:
def buildApproximation(c1,c2,basevalue,df):
    df = df.astype('float')
    df['scaledFirst'] = df.iloc[:,0].apply(lambda x: x * c1) 
    df['scaledLast'] = df.iloc[:,1].apply(lambda x: x * c2) 
    df['approximation'] = df['scaledFirst'] + df['scaledLast'] + basevalue
    return df

In [ ]:
def compressWithError2gram(firstN,error):
    df_N_2gram = df2gram.head(firstN)
    
    result = []
    all = []
    sum = []
    for ngram in df_N_2gram.index.values:
        coef,intercept,dfAprox = MLR(ngram)
        c1,c2 = coef
        df = buildApproximation(c1,c2,intercept,dfAprox)
        dfOriginal = dd.DataFrame()
        dfOriginal['values'] = df2gram.loc[ngram]
        dfOriginal['values'] = dfOriginal['values'].apply(pd.to_numeric, errors='coerce')
        #dfOriginal= pd.to_numeric(dfOriginal)
        dfOriginal['zscore'] = zscore(dfOriginal)
        df['zscore'] = zscore(df['approximation'])
        
        sum.append(dd.to_numeric(dfOriginal['values']).sum()) 
        
        rmse = mean_squared_error(dfOriginal['zscore'], df['zscore'], squared = False)
        #rmse = mean_squared_error(dfOriginal, df['approximation'])
        if rmse <= error:
            result.append([ngram,rmse,dfOriginal['values'],dfOriginal['zscore'],df['approximation'],df['zscore']]) 
        all.append(rmse)    
    return result, all, sum

In [ ]:
result, all, sum = compressWithError2gram(10000,0.05)
print(len(result))

AttributeError: 'tuple' object has no attribute 'split'

In [ ]:
graph = 1

plt.ticklabel_format(style = 'plain')
result[graph][2].plot()
plt.show()
plt.ticklabel_format(style = 'plain')
result[graph][4].plot()
plt.show()
result[graph][3].plot()
plt.show()
result[graph][5].plot()
plt.show()

print(result[graph][1])
print(result[graph][0])